In [ ]:
# Import packages 
import os
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile
from scipy.interpolate import LinearNDInterpolator as Li
from scipy.interpolate import NearestNDInterpolator as Ni
from scipy.spatial import Delaunay as Dl

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import importlib
import glob
import copy

In [ ]:
# ----------------------------------
# Functions to calculate area of
# triangle on a sphere
#-----------------------------------
def latlong_to_cartesian(latlong):
    phi   = np.deg2rad( 90 - latlong[0] )
    theta = np.deg2rad( latlong[1] )
    
    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)
    return np.array([x,y,z])
    
def spherical_area(a, b, c):
    t = np.abs( np.inner(a, np.cross(b, c) ) )
    t /= 1 + np.inner(a,b) + np.inner(b,c) + np.inner(a,c)
    return 2*np.arctan(t)
    
def flat_area(a, b, c):
    x = np.cross(a - b, c - b)
    return 0.5*np.linalg.norm(x)


In [ ]:
#scripF='/glade/p/cesmdata/inputdata/share/scripgrids/C540_SCRIP_desc.230330.nc'
scripF='PE540x3240-CF.nc4'
dS=xr.open_dataset(scripF)

In [ ]:
print(list(dS.variables))

In [ ]:
lon=dS['grid_center_lon'].values
lat=dS['grid_center_lat'].values
crlon=dS['grid_corner_lon'].values
crlat=dS['grid_corner_lat'].values
grid_area=dS['grid_area'].values


lonx0=crlon[:,0]
latx0=crlat[:,0]
lonx1=crlon[:,1]
latx1=crlat[:,1] 
lonx2=crlon[:,2]
latx2=crlat[:,2]
lonx3=crlon[:,3]
latx3=crlat[:,3] 

In [ ]:
print(np.sum(grid_area) , 4.*np.pi )

## A good place to STOP for a while

In [ ]:
refinement_c = np.sqrt( np.max(grid_area) / grid_area )
irefinement_c=np.round(refinement_c)

In [ ]:
print(np.min(irefinement_c))

In [ ]:
dScopy = copy.deepcopy( dS )
Dar1 = xr.DataArray( data=grid_area , dims=['grid_size']  , 
                    attrs=dict( description='Grid_cell_area',units='Steradians',) ,) 

dScopy['grid_area']=Dar1

Dar2 = xr.DataArray( data=irefinement_c , dims=['grid_size']  , 
                    attrs=dict( description='Integer_Refinement_Factor',units='1',) ,) 
    
dScopy['grid_refinement_factor']=Dar2

dScopy.to_netcdf( 'PE540x3240-Refinement-CF.nc' )